In [1]:
import pandas
import requests
from requests.compat import urljoin
import datetime
import time
import json

In [2]:
#Appears to be a freely available BTC API
#https://btc.com/api-doc#General

In [6]:
#Starting Point for analysis
starting_block = 501798 #Dec 31st 2017 start
ending_block = 501960 #Dec 31st 2017 end
get_blocks_by_range = 10
satoshi_to_BTC = 100000000

In [7]:
#https://chain.api.btc.com/v3/block/3
url_endpoint = "https://chain.api.btc.com"

In [8]:
#Datapoints per transaction needed
# Timestamp
# Unique Hash
# Account Sending
# Account Receiving (May be smart contract)
# Tx Fee
# Size of Data
# Network Name (duh)
# Transaction Status
# Value of Transaction

In [9]:
processed_txs = []

for b in range(get_blocks_by_range):
    next_block = starting_block + b
    next_url_call = urljoin(url_endpoint, "v3/block/" + str(next_block) + "/tx")
    resp = json.loads(requests.get(next_url_call).content)

    transactions = resp['data']['list']

    for i in transactions:
        next_tx = {'network': "Bitcoin"}

        #Unique Hash
        next_tx['txhash'] = i['hash']

        #Timestamps
        timestamp = i['block_time']
        next_tx['unixtimestamp'] = timestamp
        next_tx['datetime'] = datetime.datetime.fromtimestamp(timestamp).strftime("%Y-%m-%d %H:%M:%S")

        #Account Sending
        if i['is_coinbase']:
            next_tx['sender'] = 'Coinbase' #Can we do a numeric value to define coinbase?
        else:
            next_tx['sender'] = "SomeNumberOfSenders"

        #Account Receiving
        next_tx['receiver'] = "SomeNumberOfReceivers"

        #TxFee
        if i['is_coinbase']:
            next_tx['txfee'] = 0
        else:
            next_tx['txfee'] = i['fee']/float(satoshi_to_BTC)

        #Status
        if i['is_double_spend']:
            next_tx['status'] = 0
        else:
            next_tx['status'] = 1

        #Value of Transaction
        next_tx['value'] = i['outputs_value']/float(satoshi_to_BTC)

        processed_txs.append(next_tx)
    
    time.sleep(.5)
    
    if b%10 == 0:
        print "Current Block: " + str(next_block)
        print "Total blocks processed: " + str(b)
        print "Total Transactions processed: " + str(len(processed_txs))
        print datetime.datetime.now()

print "Completed"
print datetime.datetime.now()

Current Block: 501798
Total blocks processed: 0
Total Transactions processed: 50
2018-01-09 11:24:39.378000
Completed
2018-01-09 11:24:47.837000


In [15]:
transactions[4]

{u'block_height': 501807,
 u'block_time': 1514682087,
 u'confirmations': 1560,
 u'created_at': 1514681957,
 u'fee': 1995708,
 u'hash': u'c24606e94c15bf352d805bb5606d7ce94cd7e4e8f7eedfd204784ca4d7d71a5c',
 u'inputs': [{u'prev_addresses': [u'17A16QmavnUfCW11DAApiJxp7ARnxN5pGX'],
   u'prev_position': 30,
   u'prev_tx_hash': u'91de8c93abd76a856b2d8b2617042469e0ebf64b21f17546b57712a96772a105',
   u'prev_type': u'P2PKH',
   u'prev_value': 430978015,
   u'sequence': 4294967295L}],
 u'inputs_count': 1,
 u'inputs_value': 430978015,
 u'is_coinbase': False,
 u'is_double_spend': False,
 u'is_sw_tx': False,
 u'lock_time': 0,
 u'outputs': [{u'addresses': [u'1NsgSazAJjvZdzPNASjj6FCuzaRLUCh5HP'],
   u'spent_by_tx': u'641a68010f21ea5c1e53c3f6c75c6a109f0a5589ad91ca81d617759a94517919',
   u'spent_by_tx_position': 188,
   u'type': u'P2PKH',
   u'value': 168181},
  {u'addresses': [u'14j7eXNUhkcCD8sL1Y36WRzwDamn3m8JxV'],
   u'spent_by_tx': u'2ef613f4f28aedb713e7eefb20c7df2d8a95fd1d8bd3ebf0ffbb03bd27696288',

In [74]:
df = pandas.DataFrame.from_dict(processed_txs)

df.to_csv("Bitcoin_20171231_transactions.csv")